In [1]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG
from metric_learn import Covariance


from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn.cluster import KMeans

In [2]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, N = read_VSP_label()

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [3]:
label_visiprog = np.zeros((N,))

count = 0
for g in groups_VSP:
    label = False
    
    # check whether any entry in the group has been labelled yet
    # if yes, then merge with that existing label
    for i in g:
        if label_visiprog[i] != 0:
            label = label_visiprog[i]
            break
    
    # if no, then create a new label
    if label == False:
        count += 1
        label = count
        
    for i in g:
        label_visiprog[i] = label

print(np.max(label_visiprog))

202.0


In [4]:
model = Covariance()
model.fit(raw_feature)
X_STSIMM = model.transform(raw_feature)
# print(X_STSIMM)

In [29]:
accuracy_STSIMM = []
accuracy_mat = []
accuracy_vsp_original = []
accuracy_vsp = []

N_material = 61
N_material_train = 20

for fold in range(10):
    print("Trial {}".format(fold))

    # randomly select N_material_train material for training the metric
    # and test on the rest
    train_select = np.zeros_like(label_material).astype(bool)

    idx_train = np.array([])
    for i in range(N_material_train):
        train_class = np.random.randint(0, N_material)
        train_select |= (label_material == train_class)

    idx_train = np.where(train_select == True)[0]
    idx_test = np.where(train_select == False)[0]

    idx_train = idx_train.astype(int)
    idx_test = idx_test.astype(int)

    print("Training shape {} and test shape {}".format(len(idx_train), len(idx_test)))

    X_train = raw_feature[idx_train,:]
    X_STSIMM_train = X_STSIMM[idx_train,:]
    Y_VSP_train = label_visiprog[idx_train]
    Y_mat_train = label_material[idx_train]

    X_test = raw_feature[idx_test,:]
    X_STSIMM_test = X_STSIMM[idx_test,:]
    Y_VSP_test = label_visiprog[idx_test]
    Y_mat_test = label_material[idx_test]

    res_STSIMM = leave_one_sample_out(X_STSIMM_test, Y_mat_test)
    accuracy_STSIMM.append(res_STSIMM['accuracy'])
    print(res_STSIMM['prediction'])
    print(Y_mat_test)

    model = LFDA()
    try:
        model.fit(X_train, Y_mat_train)
        X_mat_test = model.transform(X_test)
        res = leave_one_sample_out(X_mat_test, Y_mat_test)
        accuracy_mat.append(res['accuracy'])
    except Exception as e:
        acuracy_mat.append(NaN)

    model_vsp_original = LFDA()
    try:
        model_vsp_original.fit(X_train, Y_VSP_train)
        X_vsp_original_test = model_vsp_original.transform(X_test)
        res_vsp_original = leave_one_sample_out(X_vsp_original_test, Y_mat_test)
        accuracy_vsp_original.append(res_vsp_original['accuracy'])
    except Exception as e:
        accuracy_vsp_original.append(NaN)

    model_vsp = LFDA_VISIPROG()
    try:
        model_vsp.fit(X_train, Y_VSP_train)
        X_vsp_test = model_vsp.transform(X_test)
        res_vsp = leave_one_sample_out(X_vsp_test, Y_mat_test)
        accuracy_vsp.append(res_vsp['accuracy'])
    except Exception as e:
        accuracy_vsp.append(NaN)

Trial 0
Training shape 1103 and test shape 4142
[21  0  0 ..., 60 60 60]
[ 0  0  0 ..., 60 60 60]
Trial 1
Training shape 1503 and test shape 3742
[ 5  5  5 ..., 59 59 58]
[ 5  5  5 ..., 59 59 59]
Trial 2
Training shape 1655 and test shape 3590
[21  0  0 ..., 60 60 60]
[ 0  0  0 ..., 60 60 60]
Trial 3
Training shape 1408 and test shape 3837
[15  1  1 ..., 60 60 60]
[ 1  1  1 ..., 60 60 60]
Trial 4
Training shape 1412 and test shape 3833
[15  1  1 ..., 60 60 60]
[ 1  1  1 ..., 60 60 60]
Trial 5
Training shape 1472 and test shape 3773
[21  0  0 ..., 60 60 60]
[ 0  0  0 ..., 60 60 60]
Trial 6
Training shape 1471 and test shape 3774
[21  0  0 ..., 60 60 60]
[ 0  0  0 ..., 60 60 60]
Trial 7
Training shape 1538 and test shape 3707
[21  0  0 ..., 58 58 58]
[ 0  0  0 ..., 58 58 58]
Trial 8
Training shape 1598 and test shape 3647
[15  1  1 ..., 60 60 60]
[ 1  1  1 ..., 60 60 60]
Trial 9
Training shape 1527 and test shape 3718
[51  2  2 ..., 60 60 60]
[ 2  2  2 ..., 60 60 60]


In [30]:
print(len(accuracy_mat))
print(len(accuracy_vsp))
print(len(accuracy_STSIMM))
print(len(accuracy_vsp_original))

print(res_STSIMM['accuracy'])
print(accuracy_STSIMM)

10
10
10
10
0.880043033889
[0.88701110574601638, 0.87974345269909138, 0.91476323119777159, 0.88532707844670311, 0.87946777980693969, 0.88285184203551548, 0.8778484366719661, 0.88400323711896411, 0.89361118727721411, 0.8800430338891877]


In [35]:
df = pd.DataFrame({'visiprog':accuracy_vsp, 'material':accuracy_mat, 'STSIMM':accuracy_STSIMM, \
       'visiprog_original':accuracy_vsp_original}, columns=['STSIMM','material','visiprog_original','visiprog'])

df.loc['average'] = df.mean()

display(df)
# display(df.mean())

,STSIMM,material,visiprog_original,visiprog
0,0.887011,0.979720,0.975374,0.970546
1,0.879743,0.986104,0.977819,0.984768
2,0.914763,0.983565,0.982173,0.985794
3,0.885327,0.984363,0.972635,0.983842
4,0.879468,0.988521,0.980694,0.983564
5,0.882852,0.984893,0.973496,0.983037
6,0.877848,0.976153,0.960784,0.981982
7,0.884003,0.990558,0.947936,0.987861
8,0.893611,0.986016,0.976145,0.980806
9,0.880043,0.973642,0.981442,0.975524


In [ ]:
print(model.transformer())